In [1]:
!pip install lightgbm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
# import xgboost as xgb
import lightgbm as lgb
from tqdm.notebook import tqdm

# from xgboost import plot_importance, plot_tree

# from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import auc, precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
# from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import random

plt.style.use('fivethirtyeight')

In [2]:
!cat ~/user_state

# ex.
# Now in use: ccc
# Start time: 03/31 12:00
# Estimated end time(option): 03/31 18:00

Now in use: 
Start time:  
Estimated end time(option):  

Next one who want to use: 
Estimated start time:
Estimated end time(option): 



In [3]:
!nvidia-smi

Mon Jun 26 02:04:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090         On | 00000000:01:00.0 Off |                  Off |
| 30%   48C    P2               96W / 480W|   2484MiB / 24564MiB |     12%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Time sequence length and prediction time length
seed = 55688
time_seq = 20
predict_t = 10
valid_ratio = 0.2
task = 'forecast'

In [5]:
# Setup seed
random.seed(seed)

# Get GPU
device_count = torch.cuda.device_count()
num_of_gpus = device_count

for i in range(device_count):
    print("GPU {}: {}".format(i, torch.cuda.get_device_name(i)))
    gpu_id = i

# Save best model to 
save_path = "/home/wmnlab/Documents/sheng-ru/YU/model"

# Define DataSet
dirname = "/home/wmnlab/Documents/sheng-ru/data/single"
# dirname = "/home/wmnlab/Documents/sheng-ru/data/single0.5"
dir_list = os.listdir(dirname)
dir_list = [f for f in dir_list if ( f.endswith('.csv') and (not 'sm' in f) ) ]

train_dates = ['03-26', '04-01']
test_dates = ['04-10']

def days_in_file(file, dates):
    
    for date in dates:
        if date in file: return True 
    return False

def train_valid_split(L, valid_size=0.2):
    
    length = len(L)
    v_num = int(length*valid_size)
    v_files = random.sample(L, v_num)
    t_files = list(set(L) - set(v_files))
    
    return t_files, v_files
    
# train_dir_list = [f for f in dir_list if ( f.endswith('.csv') and ('All' in f) and days_in_file(f, train_dates) )]
# test_dir_list = [f for f in dir_list if ( f.endswith('.csv') and ('All' in f) and days_in_file(f, test_dates) )]

train_dir_list, test_dir_list = train_valid_split(dir_list, valid_ratio)
train_dir_list += [f for f in os.listdir(dirname) if 'sm' in f]

# features = ['LTE_HO', 'MN_HO', 'eNB_to_ENDC', 'gNB_Rel', 'gNB_HO', 'RLF', 'SCG_RLF',
#         'num_of_neis', 'RSRP', 'RSRQ', 'RSRP1', 'RSRQ1', 'RSRP2', 'RSRQ2',
#         'nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1', 'nr-RSRP2', 'nr-RSRQ2' ]
features = ['LTE_HO', 'MN_HO', 'eNB_to_ENDC', 'gNB_Rel', 'gNB_HO', 'RLF', 'SCG_RLF',
        'num_of_neis', 'RSRP', 'RSRQ', 'RSRP1', 'RSRQ1','nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1']
# features = ['LTE_HO', 'MN_HO', 'eNB_to_ENDC', 'gNB_Rel', 'gNB_HO', 'RLF', 'SCG_RLF',
#         'num_of_neis', 'RSRP', 'RSRQ', 'RSRP1', 'RSRQ1', 'RSRP2', 'RSRQ2']

num_of_features = len(features)

target = ['LTE_HO', 'MN_HO'] # For eNB HO.
# target = ['eNB_to_ENDC'] # Setup gNB
# target = ['gNB_Rel', 'gNB_HO'] # For gNB HO.
# target = ['RLF'] # For RLF
# target = ['SCG_RLF'] # For scg failure
# target = ['dl-loss'] # For DL loss
# target = ['ul-loss'] # For UL loss

def ts_array_create(dirname, dir_list, time_seq):
    columns = ['RSRP', 'RSRQ', 'RSRP1', 'RSRQ1', 'RSRP2', 'RSRQ2',
               'nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1', 'nr-RSRP2', 'nr-RSRQ2']
    def reamin_HO_time(y_train):
        def f(L):    
            for i, e in enumerate(L):
                if e: return i+1
            return 0

        out = []
        for a2 in y_train:
            a1_out = []
            for a1 in a2:
                a1_out.append(a1.any())
      
            out.append(f(a1_out))
        return out
    
    def HO(y_train):
        out = []
        for a2 in y_train:
            if sum(a2.reshape(-1)) == 0: ho = 0
            elif sum(a2.reshape(-1)) > 0: ho = 1
            out.append(ho)
        return out

    split_time = []
    for i, f in enumerate(tqdm(dir_list)):
    
        f = os.path.join(dirname, f)
        df = pd.read_csv(f)

        # preprocess data with ffill method
        del df['Timestamp'], df['lat'], df['long'], df['gpsspeed']
        # df[columns] = df[columns].replace(0,np.nan)
        # df[columns] = df[columns].fillna(method='ffill')
        # df.dropna(inplace=True)
        
        df.replace(np.nan,0); df.replace('-',0)
        
        X = df[features]
        Y = df[target]

        Xt_list = []
        Yt_list = []

        for j in range(time_seq):
            X_t = X.shift(periods=-j)
            Xt_list.append(X_t)
    
        for j in range(time_seq,time_seq+predict_t):
            Y_t = Y.shift(periods=-(j))
            Yt_list.append(Y_t)

        # YY = Y.shift(periods=-(0))

        X_ts = np.array(Xt_list); X_ts = np.transpose(X_ts, (1,0,2)); X_ts = X_ts[:-(time_seq+predict_t-1),:,:];
        Y_ts = np.array(Yt_list); Y_ts = np.transpose(Y_ts, (1,0,2)); Y_ts = Y_ts[:-(time_seq+predict_t-1),:,:];
        split_time.append(len(X_ts))

        if i == 0:
            X_final = X_ts
            Y_final = Y_ts
        else:
            X_final = np.concatenate((X_final,X_ts), axis=0)
            Y_final = np.concatenate((Y_final,Y_ts), axis=0)

    split_time = [(sum(split_time[:i]), sum(split_time[:i])+x) for i, x in enumerate(split_time)]

    if task == 'classification':
        return X_final, HO(Y_final), split_time # classification
    elif task == 'forecast':
        return X_final, reamin_HO_time(Y_final), split_time # forecast HO
#         return X_final, np.sum(Y_final, axis=1), split_time # forecase loss

print('Loading training data...')
X_train, y_train, split_time_train = ts_array_create(dirname, train_dir_list, time_seq)
X_train_2d = X_train.reshape(-1, time_seq*num_of_features)

print('Loading testing data...')
X_test, y_test, split_time_test = ts_array_create(dirname, test_dir_list, time_seq)
X_test_2d = X_test.reshape(-1, time_seq*num_of_features)

# train_data = xgb.DMatrix(X_train_2d, label=y_train)
# test_data = xgb.DMatrix(X_test_2d, label=y_test)

GPU 0: NVIDIA GeForce RTX 4090
Loading training data...


  0%|          | 0/148 [00:00<?, ?it/s]

Loading testing data...


  0%|          | 0/29 [00:00<?, ?it/s]

In [6]:
# X_train.shape, X_train_2d.shape

In [7]:
# train_dir_list, test_dir_list

In [8]:
# Parameters
# if task == 'forecast':
#     params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", 'max_depth': 20, 'eval_metric': ['rmse', 'mae'], 'seed': seed}
# elif task == 'classification':
#     params = {'objective': 'binary:logistic', "tree_method": "gpu_hist", 'max_depth': 20, 'eta': 0.1,
#               'seed': seed,'eval_metric': ['error','logloss','aucpr']}
# n = 600

In [9]:
# evals_result = {}
# model = xgb.train(
#     params=params,
#     dtrain=train_data,
#     num_boost_round=n,
#     evals=[(train_data, 'train'), (test_data, 'valid')],
#     evals_result=evals_result,
#     early_stopping_rounds=5,
#     verbose_eval=False
# )


In [10]:
# # plot evaluation metrics
# epochs = len(evals_result['train']['error'])
# x_axis = range(0, epochs)

# fig, ax = plt.subplots()
# ax.plot(x_axis, evals_result['train']['error'], label='Train')
# ax.plot(x_axis, evals_result['valid']['error'], label='valid')
# ax.legend()
# plt.ylabel('Error')
# plt.title('XGBoost Error')

# # plot log loss
# fig, ax = plt.subplots()
# ax.plot(x_axis, evals_result['train']['logloss'], label='Train')
# ax.plot(x_axis, evals_result['valid']['logloss'], label='valid')
# ax.legend()
# plt.ylabel('Log Loss')
# plt.title('XGBoost Log Loss')

# # plot log loss
# fig, ax = plt.subplots()
# ax.plot(x_axis, evals_result['train']['aucpr'], label='Train')
# ax.plot(x_axis, evals_result['valid']['aucpr'], label='valid')
# ax.legend()
# plt.ylabel('aucpr')
# plt.title('XGBoost aucpr')

# plt.show()

In [11]:
# preds = model.predict(test_data)

In [12]:
# # plot all data point on a plot
# ths = [0.2,0.3,0.4,0.5,0.6,0.7,0.8]

# if task == 'forecast':
#     rmse = round(np.sqrt(np.mean((preds-y_test)**2)), 2)
#     mae = np.mean(np.abs(preds-np.array(y_test)))
#     print(f'RMSE is {rmse}. MAE is {mae}')
# elif task == 'classification':
#     # PR and AUCPR
#     precision, recall, thresholds = precision_recall_curve(y_test, preds)
#     aucpr = auc(recall, precision)
    
#     # ROC & AUC
#     fpr, tpr, thresholds = roc_curve(y_test, preds)
#     roc_auc = auc(fpr, tpr)
    
#     # Manually count
#     for threshold in ths:
#         TP, FP, TN, FN = 0,0,0,0
#         anses = np.where(preds<threshold, 0, 1)
#         for ans, y in zip(anses, y_test):
#             if ans == y:
#                 if ans == 1: TP += 1
#                 elif ans == 0: TN +=1
#             elif ans != y:
#                 if ans == 1: FP += 1
#                 elif ans == 0: FN +=1
#         acc=(TP+TN)/(TP+TN+FP+FN)*100
#         try:
#             p, r = round(TP/(TP+FP), 2), round(TP/(TP+FN), 2)
#             f1 = round(2*(p*r)/(p+r), 2)
#         except:
#             p, r, f1 = 'DNF', 'DNF', 'DNF'
#         s = f'th: {threshold}; Acc: {acc}; precision: {p}; recall:{r}; f1:{f1}'
#         print(s)
    
#     # 初始化閥值列表和 F1 Score 列表
#     thresholds = []
#     f1_scores = []

#     # 循環遍歷閥值，計算 F1 Score
#     for threshold in range(5, 101, 5):
#         threshold /= 100
#         y_pred_threshold = [1 if pred >= threshold else 0 for pred in preds]
#         f1_scores.append(f1_score(y_test, y_pred_threshold))
#         thresholds.append(threshold)

#     # 繪製閥值與 F1 Score 的關係圖
#     plt.figure()
#     plt.plot(thresholds, f1_scores, lw=2, label='F1 Score')
#     plt.xlabel('Threshold')
#     plt.ylabel('F1 Score')
#     plt.title('Threshold-F1 Score Relationship')
#     plt.legend()
#     plt.show()
    
#     plt.figure()
#     plt.plot(recall, precision, lw=2, label='PR Curve (AUCPR = %0.2f)' % aucpr)
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.title('Precision-Recall Curve')
#     plt.legend()
#     plt.show()
    
#     print(f'ROC Auc is {roc_auc}.')
#     plt.figure()
#     plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
#     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.title(f'ROC curve')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend(loc="lower right")
#     plt.show()

In [13]:
# # 将模型预测错误的样本保存到一个列表中
# errors = []
# trues = []
# for i in range(len(y_test)):
#     if preds[i] > 0.5 and y_test[i] == 0:
#         errors.append(X_test[i])
#         trues.append(0)
#     elif preds[i] <= 0.5 and y_test[i] == 1:
#         errors.append(X_test[i])
#         trues.append(1)
# # # 保存到文件中
# # with open("error_samples.txt", "w") as f:
# #     for sample in errors:
# #         f.write(str(sample) + "\n")


In [14]:
# # visulized on many sample on validation data
# sample_value = 2
# # samples = random.sample(split_time_test, sample_value)
# samples = [split_time_test[8], split_time_test[9]]

# fig, axs = plt.subplots(1, sample_value, figsize=(14, 2.5))

# # y_test
# # preds

# for i in range(sample_value):
#     true = [y_test[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(true, label='true')
#     prediciton = [preds[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(prediciton, label='pred')

# plt.legend()
# plt.show()

In [15]:
# A = [test_dir_list[split_time_test.index(s)] for s in samples]
# A

In [16]:
# # visulized on many sample on validation data
# sample_value = 2
# # samples = random.sample(split_time_test, sample_value)
# samples = [split_time_test[0], split_time_test[1]]
# fig, axs = plt.subplots(1, sample_value, figsize=(14, 2.5))

# # y_test
# # preds

# for i in range(sample_value):
#     true = [y_test[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(true, label='true')
#     prediciton = [preds[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(prediciton, label='pred')

# plt.legend()
# plt.show()

In [17]:
# A = [test_dir_list[split_time_test.index(s)] for s in samples]
# A

In [18]:
# # Check whethet it do good on training data
# sample_value = 2
# samples = random.sample(split_time_train, sample_value)
# preds_train = model.predict(train_data)

# fig, axs = plt.subplots(1, sample_value, figsize=(14, 2.5))

# for i in range(sample_value):
#     true = [y_train[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(true, label='true')
#     prediciton = [preds_train[i] for i in range(samples[i][0], samples[i][1])]
#     axs[i].plot(prediciton, label='pred')

# plt.legend()
# plt.show()

# grid search

## Search on task

In [24]:
def run():
    # Parameters
    if task == 'forecast':
        params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", 'max_depth': 20, 'eval_metric': ['rmse', 'mae'], 'seed': seed}
    elif task == 'classification':
        params = {'objective': 'binary:logistic', "tree_method": "gpu_hist", 'eta': lr, 'max_depth': max_d,
                  'subsample': subsample, 'colsample_bytree': colsample_bytree,
                  'alpha': reg_alpha,'gamma': gamma,  
                  'seed': seed,'eval_metric': ['error','logloss']}

    model = lgb.train(
        params=params,
        data=train_data,
        num_boost_round=n_esti,
        evals=[(train_data, 'train'), (test_data, 'valid')],
        early_stopping_rounds=10,
        verbose_eval=False
    )

    preds = model.predict(test_data)

    # plot all data point on a plot
    ths = [0.2,0.3,0.4,0.5,0.6,0.7,0.8]

    if task == 'forecast':
        rmse = round(np.sqrt(np.mean((preds-y_test)**2)), 2)
        plt.title(f'RMSE is {rmse}.')
    elif task == 'classification':
        # PR and AUCPR
        precision, recall, thresholds = precision_recall_curve(y_test, preds)
        aucpr = auc(recall, precision)
        
        # ROC & AUC
        fpr, tpr, thresholds = roc_curve(y_test, preds)
        roc_auc = auc(fpr, tpr)
        
        threshold = 0.5
        TP, FP, TN, FN = 0,0,0,0
        anses = np.where(preds<threshold, 0, 1)
        for ans, y in zip(anses, y_test):
            if ans == y:
                if ans == 1: TP += 1
                elif ans == 0: TN +=1
            elif ans != y:
                if ans == 1: FP += 1
                elif ans == 0: FN +=1
        acc=(TP+TN)/(TP+TN+FP+FN)*100
        try:
            p, r = round(TP/(TP+FP), 2), round(TP/(TP+FN), 2)
            f1 = round(2*(p*r)/(p+r), 2)
        except:
            p, r, f1 = 'DNF', 'DNF', 'DNF'
        s = f'th: {threshold}; Acc: {acc}; precision: {p}; recall:{r}; f1:{f1}'
        print(s)
        
        print('PR Curve (AUCPR = %0.2f)' % aucpr)
        print(f'ROC Auc is {roc_auc}.')
        print('============================================================')
        cols_out = [time_seq, predict_t, lr, max_d, n_esti, subsample, colsample_bytree, reg_alpha, gamma, roc_auc, aucpr, p, r, f1]
        cols_out = [str(n) for n in cols_out]
        f_out.write(','.join(cols_out)+'\n')
        
# Time sequence length and prediction time length
seed = 55688
time_seqs = [20]
predict_ts = list(range(1,20+1,2))
lrs = [0.3, 0.1]
max_ds = [7, 15]
n_estis = [200]
subsamples = [1]
colsample_bytrees = [0.9]
reg_alphas = [0.5]
gammas = [0.1]
valid_ratio = 0.2
task = 'classification'

dirname = "/home/wmnlab/Documents/sheng-ru/data/single"
dir_list = os.listdir(dirname)
dir_list = [f for f in dir_list if ( f.endswith('.csv') and (not 'sm' in f) ) ]

train_dir_list, test_dir_list = train_valid_split(dir_list, valid_ratio)
train_dir_list += [f for f in os.listdir(dirname) if 'sm' in f]


features = ['LTE_HO', 'MN_HO', 'eNB_to_ENDC', 'gNB_Rel', 'gNB_HO', 'RLF', 'SCG_RLF',
            'num_of_neis', 'RSRP', 'RSRQ', 'RSRP1', 'RSRQ1','nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1']
target = ['LTE_HO', 'MN_HO'] # For eNB HO.
# target = ['eNB_to_ENDC']
# target = ['RLF']

num_of_features = len(features)

out = 'result_tpreds_lte_HO.csv'
f_out = open(out, mode='w')
cols_out = ['t_seq','t_pred','lr','max_dep','n_esti','subsample','colsample_bytree','reg_alpha','gamma','auc','aucpr','p','r','f1']
f_out.write(','.join(cols_out)+'\n')

train_pos = []
train_neg = []

for time_seq in time_seqs:
    for predict_t in predict_ts:
        print(f'predict_t {predict_t}')
        X_train, y_train, split_time_train = ts_array_create(dirname, train_dir_list, time_seq)
        X_train_2d = X_train.reshape(-1, time_seq*num_of_features)

        X_test, y_test, split_time_test = ts_array_create(dirname, test_dir_list, time_seq)
        X_test_2d = X_test.reshape(-1, time_seq*num_of_features)

        train_data = lgb.Dataset(X_train, y_train)
        test_data = lgb.Dataset(X_test, y_test)
        
        train_neg.append(y_train.count(0)); train_pos.append(y_train.count(1))
        print('train 0:',y_train.count(0), 'train 1:', y_train.count(1))
        
        for lr in lrs:
            for max_d in max_ds:
                for n_esti in n_estis:
                    for subsample in subsamples:
                        for colsample_bytree in colsample_bytrees:
                            for reg_alpha in reg_alphas:
                                for gamma in gammas:
                                    run()

f_out.close()

predict_t 1


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

train 0: 37818 train 1: 2384


TypeError: train() got an unexpected keyword argument 'data'

In [ ]:
train_pos, train_neg
y = []
x = list(range(1,20+1))
for p, n in zip(train_pos, train_neg):
    y.append(p/(p+n))

ax = plt.gca()
x_ticks = range(0,20+1,2)
ax.set_xticks(x_ticks)
plt.title('Data Balance')
plt.plot(x, y, label='Positive Ratio')
plt.xlabel('Prediction Time')
plt.legend()
plt.show()

In [ ]:
# df = pd.read_csv('result_tpreds_SN_setup.csv')
df = pd.read_csv('result_tpreds_SN_setup.csv')
x = df["t_pred"]
y1 = df["auc"]
y2 = df["aucpr"]
plt.plot(x, y1, label='Auc')
plt.plot(x, y2, label='Aucpr')
ax = plt.gca()
x_ticks = np.arange(2,22,2)
ax.set_xticks(x_ticks)
plt.xlabel(r'$\bf Predict\ Time\ (sec)$')
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv('result_tseqs.csv')
x = df["t_seq"]
y1 = df["auc"]
y2 = df["aucpr"]
plt.plot(x, y1, label='Auc')
plt.plot(x, y2, label='Aucpr')
plt.xlabel(r'$\bf Input\ Length\ (sec)$')
ax = plt.gca()
x_ticks = np.arange(2,22,2)
ax.set_xticks(x_ticks)
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv('result_predt.csv')
x = df["t_pred"]
y1 = df["auc"]
y2 = df["aucpr"]
plt.plot(x, y1, label='Auc')
plt.plot(x, y2, label='Aucpr')
ax = plt.gca()
x_ticks = np.arange(2,22,2)
ax.set_xticks(x_ticks)
plt.xlabel(r'$\bf Predict\ Time\ (sec)$')
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv('result.csv')
max_value = df['aucpr'].max()
max_index = df['aucpr'].idxmax()
df.iloc[max_index]
max_value = df['auc'].max()
max_index = df['auc'].idxmax()
df.iloc[max_index]

In [ ]:
X = 'gamma'
y = [0.,0.1,0.5]
df = pd.read_csv('result.csv')
x = list(range(729))
A = df[df[X]==y[0]]
B = df[df[X]==y[1]]
C = df[df[X]==y[2]]
plt.plot(x,A['aucpr'], label='1')
plt.plot(x,B['aucpr'], label='2')
plt.plot(x,C['aucpr'], label='3')
# plt.plot(df['auc'], label='auc')
plt.legend()
plt.show()

# Stage II. Classification and Forecast

In [ ]:
def ts_array_create(dirname, dir_list, time_seq):
    
    columns = ['RSRP', 'RSRQ', 'RSRP1', 'RSRQ1', 'RSRP2', 'RSRQ2',
               'nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1', 'nr-RSRP2', 'nr-RSRQ2']
    
    def reamin_HO_time(y_train):
        def f(L):    
            for i, e in enumerate(L):
                if e: return i+1
            return 0

        out = []
        for a2 in y_train:
            a1_out = []
            for a1 in a2:
                a1_out.append(a1.any())
      
            out.append(f(a1_out))
        return out
    
    def HO(y_train):
        out = []
        for a2 in y_train:
            if sum(a2.reshape(-1)) == 0: ho = 0
            elif sum(a2.reshape(-1)) > 0: ho = 1
            out.append(ho)
        return out

    split_time = []
    for i, f in enumerate(tqdm(dir_list)):
    
        f = os.path.join(dirname, f)
        df = pd.read_csv(f)

        # preprocess data with ffill method
        del df['Timestamp'], df['lat'], df['long'], df['gpsspeed']
        # df[columns] = df[columns].replace(0, np.nan)
        # df[columns] = df[columns].fillna(method='ffill')
        # df.dropna(inplace=True)
        
        df.replace(np.nan,0,inplace=True); df.replace('-',0,inplace=True)
        
        X = df[features]
        Y = df[target]

        Xt_list = []
        Yt_list = []

        for j in range(time_seq):
            X_t = X.shift(periods=-j)
            Xt_list.append(X_t)
    
        for j in range(time_seq,time_seq+predict_t):
            Y_t = Y.shift(periods=-(j))
            Yt_list.append(Y_t)

        # YY = Y.shift(periods=-(0))

        X_ts = np.array(Xt_list); X_ts = np.transpose(X_ts, (1,0,2)); X_ts = X_ts[:-(time_seq+predict_t-1),:,:]
        Y_ts = np.array(Yt_list); Y_ts = np.transpose(Y_ts, (1,0,2)); Y_ts = Y_ts[:-(time_seq+predict_t-1),:,:]
        split_time.append(len(X_ts))

        if i == 0:
            X_final = X_ts
            Y_final = Y_ts
        else:
            X_final = np.concatenate((X_final,X_ts), axis=0)
            Y_final = np.concatenate((Y_final,Y_ts), axis=0)

    split_time = [(sum(split_time[:i]), sum(split_time[:i])+x) for i, x in enumerate(split_time)]
    
    return X_final, np.array(HO(Y_final)), np.array(reamin_HO_time(Y_final)), split_time # forecast HO

## Classification

In [ ]:
# Time sequence length and prediction time length
seed = 55688
time_seq = 20
predict_t = 20
lr = 0.1
max_d = 15
n_esti = 200
subsample = 1
colsample_bytree = 0.9
reg_alpha = 0.5
gamma = 0.1
valid_ratio = 0.2

random.seed(seed)

dirname = "/home/wmnlab/Documents/sheng-ru/data/single"
dir_list = os.listdir(dirname)
dir_list = [f for f in dir_list if ( f.endswith('.csv') and (not 'sm' in f) ) ] # Take out data with sm
# dir_list = [f for f in dir_list if ( f.endswith('.csv') and (not '_B1.' in f and not '_LTE' in f) )] # Take out B1 and LTE not NSA data

train_dir_list, test_dir_list = train_valid_split(dir_list, valid_ratio)
train_dir_list += [f for f in os.listdir(dirname) if 'sm' in f] # Put sm data in train dir
# train_dir_list = [f for f in train_dir_list if (not '_B1.' in f and not '_LTE' in f)] # Take out sm data of not NSA mode 

features = ['LTE_HO', 'MN_HO', 'eNB_to_ENDC', 'gNB_Rel', 'gNB_HO', 'RLF', 'SCG_RLF',
            'num_of_neis', 'RSRP', 'RSRQ', 'RSRP1', 'RSRQ1',
            'nr-RSRP', 'nr-RSRQ', 'nr-RSRP1', 'nr-RSRQ1']
# target = ['LTE_HO', 'MN_HO'] # For eNB HO.
target = [ 'gNB_Rel', 'gNB_HO']
# target = ['eNB_to_ENDC'] # For SN setup
# target = ['RLF']

num_of_features = len(features)

# Data
X_train, y_train1, y_train2, split_time_train = ts_array_create(dirname, train_dir_list, time_seq)
X_train_2d = X_train.reshape(-1, time_seq*num_of_features)

X_test, y_test1, y_test2, split_time_test = ts_array_create(dirname, test_dir_list, time_seq)
X_test_2d = X_test.reshape(-1, time_seq*num_of_features)

train_data = xgb.DMatrix(X_train_2d, label=y_train1)
test_data = xgb.DMatrix(X_test_2d, label=y_test1)


In [ ]:
# Parameters
params = {'objective': 'binary:logistic', "tree_method": "gpu_hist", 'eta': lr, 'max_depth': max_d,
            'subsample': subsample, 'colsample_bytree': colsample_bytree, 'alpha': reg_alpha,'gamma': gamma,  
            'seed': seed,'eval_metric': ['error','logloss','aucpr']}

model1 = xgb.train(
    params=params,
    dtrain=train_data,
    num_boost_round=n_esti,
    evals=[(train_data, 'train'), (test_data, 'valid')],
    early_stopping_rounds=10,
    verbose_eval=False
)

In [ ]:
preds1 = model1.predict(test_data)

# plot all data point on a plot
ths = [0.2,0.3,0.4,0.5,0.6,0.7,0.8]

# PR and AUCPR
precision, recall, thresholds = precision_recall_curve(y_test1, preds1)
aucpr = auc(recall, precision)

# ROC & AUC
fpr, tpr, thresholds = roc_curve(y_test1, preds1)
roc_auc = auc(fpr, tpr)

# Record Error 
Errors = []


# Manually count
for threshold in ths:
    TP, FP, TN, FN = 0,0,0,0
    anses = np.where(preds1<threshold, 0, 1)
    for ans, y, y2 in zip(anses, y_test1, y_test2):
        if ans == y:
            if ans == 1: TP += 1
            elif ans == 0: TN +=1
        elif ans != y:
            Errors.append(y2)
            if ans == 1: FP += 1
            elif ans == 0: FN +=1
    acc=(TP+TN)/(TP+TN+FP+FN)*100
    try:
        p, r = round(TP/(TP+FP), 2), round(TP/(TP+FN), 2)
        f1 = round(2*(p*r)/(p+r), 2)
    except:
        p, r, f1 = 'DNF', 'DNF', 'DNF'
    s = f'th: {threshold}; Acc: {acc}; precision: {p}; recall:{r}; f1:{f1}'
    print(s)
print('PR Curve (AUCPR = %0.2f)' % aucpr)
print(f'ROC Auc is {roc_auc}.')
print('==========================================================================')


In [ ]:
# save model
save_path = '/home/wmnlab/Documents/sheng-ru/YU/model/'
save_name = 'setup_cls_xgb.json'
f = os.path.join(save_path, save_name)
model1.save_model(f)

In [ ]:
# Time sequence length and prediction time length
lr = 0.3
max_d = 7
n_esti = 200
subsample = 1
colsample_bytree = 0.9
reg_alpha = 0.5
gamma = 0.1
task = 'forecast'

cond = y_train2 > 0
X_train_2d_fore = X_train_2d[cond]
y_train2_fore = y_train2[cond]
cond = y_test2 > 0
X_test_2d_fore = X_test_2d[cond]
y_test2_fore = y_test2[cond]

train_data = xgb.DMatrix(X_train_2d_fore, label=y_train2_fore)
test_data = xgb.DMatrix(X_test_2d_fore, label=y_test2_fore)

In [ ]:
# Parameters
if task == 'forecast':
    params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", 'eta': lr, 'max_depth': max_d,
              'subsample': subsample, 'colsample_bytree': colsample_bytree, 'alpha': reg_alpha,'gamma': gamma,  
              'seed': seed,'eval_metric': ['rmse', 'mae']}
elif task == 'classification':
    params = {'objective': 'binary:logistic', "tree_method": "gpu_hist", 'eta': lr, 'max_depth': max_d,
                'subsample': subsample, 'colsample_bytree': colsample_bytree, 'alpha': reg_alpha,'gamma': gamma,  
                'seed': seed,'eval_metric': ['error','logloss','aucpr']}

model2 = xgb.train(
    params=params,
    dtrain=train_data,
    num_boost_round=600,
    evals=[(train_data, 'train'), (test_data, 'valid')],
    early_stopping_rounds=10,
    # verbose_eval=False
)

In [ ]:
preds2 = model2.predict(test_data)

In [ ]:
plt.figure(figsize=(8, 6))
x = 400
y = 500
plt.plot(preds2[x:y], label = 'preds')
plt.plot(y_test2_fore[x:y], label = 'true')
plt.ylim([0,predict_t])
plt.legend()
plt.show()

## Visualization

In [ ]:
test_data = xgb.DMatrix(X_test_2d)
preds_ho = model1.predict(test_data)
preds_ho_time = model2.predict(test_data)

In [ ]:
ans = np.where(preds_ho>0.5, preds_ho_time, 0)

In [ ]:
# visulized on many sample on validation data
sample_value = 2
# samples = random.sample(split_time_test, sample_value)
samples = [split_time_test[0], split_time_test[1]]

fig, axs = plt.subplots(1, sample_value, figsize=(14, 2.5))

for i in range(sample_value):
    
    true_ho_time = [y_test2[i] for i in range(samples[i][0], samples[i][1])]
        
    axs[i].plot(true_ho_time, label='true')
    prediciton_ho = [ans[i] for i in range(samples[i][0], samples[i][1])]
    axs[i].plot(prediciton_ho, label='pred')

plt.legend()
plt.show()

In [ ]:
# save model
save_path = '/home/wmnlab/Documents/sheng-ru/YU/model/'
save_name = 'nr_HO_fcst_xgb.json'
f = os.path.join(save_path, save_name)
model2.save_model(f)